In [1]:
# coding: utf-8
# Computer Vision API

# In[1]:
from __future__ import print_function
import bing_voice
from bing_voice import *
import pyaudio
import sys
import wave
import os
from bottle import route, run
from bottle import template
import time 
import requests
import cv2
import operator
import numpy as np
# Import library to display results
import matplotlib.pyplot as plt
from bottle import request
from bottle import static_file,get
import urllib
#os.chdir("C:\Indira\Code\BottleApp")



In [2]:
_url = 'https://westus.api.cognitive.microsoft.com/vision/v1.0/analyze?'
_key = 'ddd4c6b08a344f8494939e814aa45339'
_maxNumRetries = 10  
@get("/static/img/<filepath:re:.*\.(jpg|png|gif|ico|svg)>")
def img(filepath):
    return static_file(filepath, root="static/images/work/")

@get("/tmp/upload/<filepath:re:.*\.(jpg|png|gif|ico|svg)>")
def img(filepath):
    return static_file(filepath, root="tmp/upload/")

    
@get("/static/css/<filepath:re:.*\.css>")
def css(filepath):
    return static_file(filepath, root="static/css/")

@get("/static/js/<filepath:re:.*\.js>")
def js(filepath):
    return static_file(filepath, root="static/js/")



In [3]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None
    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['message'] ) )
            
            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['message'] ) )
            print("Here")
        break
        
    return result


In [4]:
def renderResultOnImage( result, img ):
   
    """Display the obtained results onto the input image"""

    R = int(result['color']['accentColor'][:2],16)
    G = int(result['color']['accentColor'][2:4],16)
    B = int(result['color']['accentColor'][4:],16) ;
    cv2.rectangle( img,(0,0), (img.shape[1], img.shape[0]), color = (R,G,B), thickness = 25 )

    if 'description' in result:
        desc = result['description']['captions'][0]['text']
        cv2.putText( img, desc, (100,200), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (73,25,180), 2 )
        print(desc)


In [5]:
@route('/edit')
def edit():
    info = []
    return template('simple.tpl', info)


@route('/vision')
def vision():
    
    datadic = {        
        'category ' : "",
        'description' : "",
        'tags' : list()
    }
    return(template("vision.tpl",datadic = datadic,img = "/static/img/default.png"))


In [9]:
@route('/vision/show', method = 'POST')
def upload():
       
    #p = os.path.join(os.getcwd(),"tmp/upload/", upload.filename)
        
    #p = os.path.join(os.getcwd(),"tmp/upload/", upload.filename)
    #upload.save(p)
       
    upload = request.files.get('upload')
    name, ext = os.path.splitext(upload.filename)
    if ext not in ('.png', '.jpg', '.jpeg'):
        return "File extension not allowed."

    save_path = "tmp/upload"
    if not os.path.exists(save_path):
        os.makedirs(save_path)

   
    
    file_path = "{path}/{file}".format(path=save_path, file=upload.filename)
    try:
        os.remove(file_path)
    except Exception as e:
        pass
    upload.save(file_path)
    
    with open( file_path, 'rb' ) as f:
        data = f.read()
   
    # Computer Vision parameters
    params = { 'visualFeatures' : 'Color,Categories,Faces,Description'} 

    headers = {}
    headers['Ocp-Apim-Subscription-Key'] = _key
    #headers['Content-Type'] = 'application/json' 
    headers['Content-Type'] = 'application/octet-stream'
    json = None
    #json = { 'url': urlImage } 
    #data = None
    res = processRequest( json, data, headers, params)
    print(res)
       
    if res is not None: 
        datadic = { 'category' : res['categories'][0]['name']  , 'description': res['description']['captions'][0]['text'] ,
                   'tags':  res['description']['tags'][:5]}
        print(datadic)
        
        return template('vision.tpl',datadic=datadic, img="/"+file_path )  
 

In [10]:
run(host='localhost', port=8080, debug=True)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [28/Apr/2017 12:21:52] "GET / HTTP/1.1" 404 720
127.0.0.1 - - [28/Apr/2017 12:21:56] "GET /edit HTTP/1.1" 200 2202
127.0.0.1 - - [28/Apr/2017 12:21:56] "GET /static/images/pfd-sprites.png HTTP/1.1" 404 778
127.0.0.1 - - [28/Apr/2017 12:22:07] "GET /vision HTTP/1.1" 200 2453


{u'description': {u'captions': [{u'text': u'a colorful bird sitting on a branch', u'confidence': 0.61586676427159}], u'tags': [u'bird', u'animal', u'insect', u'sitting', u'green', u'top', u'black', u'small', u'wire', u'colorful', u'perched', u'table', u'large', u'red', u'yellow', u'branch', u'blue']}, u'color': {u'accentColor': u'43B100', u'dominantColorForeground': u'Green', u'dominantColorBackground': u'Green', u'dominantColors': [u'Green', u'Black', u'Yellow'], u'isBWImg': False}, u'requestId': u'81c20f04-de42-4ffb-b408-819e45c26462', u'faces': [], u'categories': [{u'score': 0.00390625, u'name': u'abstract_'}, {u'score': 0.52734375, u'name': u'abstract_shape'}, {u'score': 0.00390625, u'name': u'others_'}], u'metadata': {u'width': 640, u'format': u'Jpeg', u'height': 425}}
{'category': u'abstract_', 'description': u'a colorful bird sitting on a branch', 'tags': [u'bird', u'animal', u'insect', u'sitting', u'green']}


127.0.0.1 - - [28/Apr/2017 12:22:34] "POST /vision/show HTTP/1.1" 200 2585
127.0.0.1 - - [28/Apr/2017 12:22:34] "GET /tmp/upload/1.jpg HTTP/1.1" 200 119738


{u'description': {u'captions': [{u'text': u'a swan next to a lake', u'confidence': 0.7453175308398567}], u'tags': [u'outdoor', u'water', u'grass', u'lake', u'bird', u'swan', u'man', u'small', u'body', u'sitting', u'pond', u'large', u'river', u'riding', u'white', u'floating', u'standing', u'swimming', u'boat']}, u'color': {u'accentColor': u'3F7522', u'dominantColorForeground': u'Black', u'dominantColorBackground': u'Black', u'dominantColors': [u'Black', u'Grey'], u'isBWImg': False}, u'requestId': u'1e027061-b5c0-4904-91d9-b4d2dc9ff559', u'faces': [], u'categories': [{u'score': 0.0078125, u'name': u'outdoor_'}, {u'score': 0.3203125, u'name': u'outdoor_waterside'}], u'metadata': {u'width': 1600, u'format': u'Jpeg', u'height': 1200}}
{'category': u'outdoor_', 'description': u'a swan next to a lake', 'tags': [u'outdoor', u'water', u'grass', u'lake', u'bird']}


127.0.0.1 - - [28/Apr/2017 12:22:44] "POST /vision/show HTTP/1.1" 200 2567
127.0.0.1 - - [28/Apr/2017 12:22:46] "GET /tmp/upload/2.jpg HTTP/1.1" 200 1860184


...

127.0.0.1 - - [28/Apr/2017 12:23:01] "POST /vision/show HTTP/1.1" 200 2567
127.0.0.1 - - [28/Apr/2017 12:23:03] "GET /tmp/upload/2.jpg HTTP/1.1" 200 1860184
127.0.0.1 - - [28/Apr/2017 12:28:13] "GET /vision/show HTTP/1.1" 405 744


{u'description': {u'captions': [{u'text': u'a full view of a city', u'confidence': 0.39455336482031717}], u'tags': [u'outdoor', u'grass', u'building', u'filled', u'full', u'field', u'many', u'yard', u'large', u'water', u'old', u'several', u'city', u'boat', u'grazing', u'white']}, u'color': {u'accentColor': u'8F893C', u'dominantColorForeground': u'Yellow', u'dominantColorBackground': u'Yellow', u'dominantColors': [u'Yellow'], u'isBWImg': False}, u'requestId': u'7e803a23-9a7a-4022-bf7f-d6732c5e61d3', u'faces': [], u'categories': [{u'score': 0.00390625, u'name': u'abstract_'}, {u'score': 0.00390625, u'name': u'outdoor_'}], u'metadata': {u'width': 2560, u'format': u'Jpeg', u'height': 1500}}
{'category': u'abstract_', 'description': u'a full view of a city', 'tags': [u'outdoor', u'grass', u'building', u'filled', u'full']}


127.0.0.1 - - [28/Apr/2017 12:28:31] "POST /vision/show HTTP/1.1" 200 2581
127.0.0.1 - - [28/Apr/2017 12:28:31] "GET /tmp/upload/grass.jpg HTTP/1.1" 200 499276


{u'description': {u'captions': [{u'text': u'a colorful bird sitting on a branch', u'confidence': 0.61586676427159}], u'tags': [u'bird', u'animal', u'insect', u'sitting', u'green', u'top', u'black', u'small', u'wire', u'colorful', u'perched', u'table', u'large', u'red', u'yellow', u'branch', u'blue']}, u'color': {u'accentColor': u'43B100', u'dominantColorForeground': u'Green', u'dominantColorBackground': u'Green', u'dominantColors': [u'Green', u'Black', u'Yellow'], u'isBWImg': False}, u'requestId': u'0794f426-d5b3-4617-81c6-942532e1b272', u'faces': [], u'categories': [{u'score': 0.00390625, u'name': u'abstract_'}, {u'score': 0.52734375, u'name': u'abstract_shape'}, {u'score': 0.00390625, u'name': u'others_'}], u'metadata': {u'width': 640, u'format': u'Jpeg', u'height': 425}}
{'category': u'abstract_', 'description': u'a colorful bird sitting on a branch', 'tags': [u'bird', u'animal', u'insect', u'sitting', u'green']}


127.0.0.1 - - [28/Apr/2017 12:29:07] "POST /vision/show HTTP/1.1" 200 2585
127.0.0.1 - - [28/Apr/2017 12:29:07] "GET /tmp/upload/1.jpg HTTP/1.1" 200 119738


In [12]:
## Text To Speech
@route('/tts',method = "POST")
def tts():      
    # get a key from https://www.microsoft.com/cognitive-services/en-us/speech-api
    BING_KEY = '756f8cbc3ad54b209076e4e41b51ecc2'
    CHUNK_SIZE = 2048

    text   = request.forms.get('t')
  

    bing = BingVoice(BING_KEY)
    data = bing.synthesize(text)

    pa = pyaudio.PyAudio()
    stream = pa.open(format=pyaudio.paInt16,
                     channels=1,
                     rate=16000,
                     output=True,
                     # output_device_index=1,
                     frames_per_buffer=CHUNK_SIZE)

    stream.write(data)
    stream.close()

    if len(text) >= 3:
        wf = wave.open(sys.argv[2], 'wb')
        wf.setframerate(16000)
        wf.setnchannels(1)
        wf.setsampwidth(2)

        wf.writeframes(data)
        wf.close()


    #return template('tts.tpl', info)


In [13]:
@route('/tts')
def tts():      
    return(template('tts.tpl'))

In [14]:
@route('/spellcheck')
def spellcheck():      
    return(template('spellcheck.tpl',opstr = ''))

In [15]:
import  httplib
@route('/spellcheck',method = "POST")
def spellcheckpost():
    headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': 'baf9b2caff1849c08a4428bf637cbbb4',
    'Content-Type': 'application/x-www-form-urlencoded',
    }
    text   = request.forms.get('t')
    s= text
    text = "Text=" + text
    params = urllib.urlencode({
        
        'mode': 'proof',
        'mkt': 'en-us',
    })
    
    
    try:
        conn = httplib.HTTPSConnection('api.cognitive.microsoft.com')
        conn.request("POST", "/bing/v5.0/spellcheck/?%s" % params,text, headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
           print(e)
    print(s)
    jsondata = json.loads(data)
    for row in jsondata['flaggedTokens']:
        position =  row['offset']
        #print(row['token'])
        replacement = row['suggestions'][0]['suggestion'] ## suggestion with the maximum score 
        length_of_replaced= len(replacement)
        if(len(row['token']) < length_of_replaced):
               s = s[:position] + replacement + " " + s[position+length_of_replaced:]
        else:
               s = s[:position] + replacement  + s[position+length_of_replaced:]
    

    return(template('spellcheck.tpl',opstr = s))

In [ ]:
run(host='localhost', port=8080, debug=True)

Bottle v0.12.13 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [28/Apr/2017 12:30:29] "GET /visio HTTP/1.1" 404 730
127.0.0.1 - - [28/Apr/2017 12:30:33] "GET /vision HTTP/1.1" 200 2453
127.0.0.1 - - [28/Apr/2017 12:30:41] "GET /spellcheck HTTP/1.1" 200 1389
127.0.0.1 - - [28/Apr/2017 12:33:12] "GET /spellcheck HTTP/1.1" 200 1395
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/css/main.css HTTP/1.1" 200 11928
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/js/modernizr-2.6.2.js HTTP/1.1" 200 11333
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/img/header.jpg HTTP/1.1" 200 105817
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/js/pyaari-menu.1.0.js HTTP/1.1" 200 1285
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/js/pyaari-main.1.0.js HTTP/1.1" 200 916
127.0.0.1 - - [28/Apr/2017 12:33:13] "GET /static/js/jquery-1.11.0.min.js HTTP/1.1" 200 96381
127.0.0.1 - - [28/Apr/2017 12:33:32] "GET /spellcheck HTTP/1.1"

{"_type": "SpellCheck", "flaggedTokens": [{"offset": 46, "token": "sence", "type": "UnknownToken", "suggestions": [{"suggestion": "sense", "score": 0.743168381767998}]}, {"offset": 59, "token": "gys", "type": "UnknownToken", "suggestions": [{"suggestion": "guys", "score": 0.775746049735409}]}]}
 Hi this is to tst if hwte we are tying makes sence to you gys 


127.0.0.1 - - [28/Apr/2017 12:34:58] "POST /spellcheck HTTP/1.1" 200 1521
127.0.0.1 - - [28/Apr/2017 12:34:58] "GET /static/css/main.css HTTP/1.1" 304 0


{"_type": "SpellCheck", "flaggedTokens": [{"offset": 9, "token": "realy", "type": "UnknownToken", "suggestions": [{"suggestion": "really", "score": 0.82559978705741}]}, {"offset": 19, "token": "Iam", "type": "UnknownToken", "suggestions": [{"suggestion": "I'm", "score": 0.82559978705741}, {"suggestion": "I am", "score": 0.905526593140847}, {"suggestion": "Im", "score": 0.67451771107935}]}, {"offset": 23, "token": "crzy", "type": "UnknownToken", "suggestions": [{"suggestion": "crazy", "score": 0.82559978705741}]}]}
 Did you realy say Iam crzy


127.0.0.1 - - [28/Apr/2017 12:36:23] "POST /spellcheck HTTP/1.1" 200 1507
127.0.0.1 - - [28/Apr/2017 12:36:23] "GET /static/css/main.css HTTP/1.1" 304 0


In [116]:
def upload():
    upload = request.files.get('upload')
    name, ext = os.path.splitext(upload.filename)
    if ext not in ('.png', '.jpg', '.jpeg'):
        return "File extension not allowed."
        
    save_path = "/tmp/upload"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    p = os.path.join(os.getcwd(),"tmp/upload/", upload.filename)
        
    if(os.path.exists(p)) :
        os.remove(p)
    
    p = os.path.join(os.getcwd(),"tmp/upload/", upload.filename)
    upload.save(p)
       
                         
    with open( p, 'r' ) as f:
        data = f.read()
   
    # Computer Vision parameters
    params = { 'visualFeatures' : 'Color,Categories,Faces,Description'} 

    headers = {}
    headers['Ocp-Apim-Subscription-Key'] = _key
    #headers['Content-Type'] = 'application/json' 
    headers['Content-Type'] = 'application/octet-stream'
    json = None
    #json = { 'url': urlImage } 
    #data = None
    res = processRequest( json, data, headers, params)
    print(res)
       
    if res is not None: 
        datadic = { 'category' : res['categories'][0]['name']  , 'description': res['description']['captions'][0]['text'] ,
                   'tags':  res['description']['tags'][:5]}
        print(datadic)
        return template('vision.tpl',datadic=datadic, img=file_path )  
 

{'category': u'outdoor_oceanbeach',
 'description': u'a chair sitting in front of a beach',
 'tags': [u'outdoor', u'water', u'beach', u'sunset', u'ocean']}

In [115]:
dict = {'category' : res['categories'][0]['name']  , 'description': res['description']['captions'][0]['text'] , 'tags':  res['description']['tags'][:5] }

In [92]:
res = upload()

In [95]:
type(res)

dict

In [96]:
res

{u'categories': [{u'name': u'outdoor_oceanbeach', u'score': 0.66796875}],
 u'color': {u'accentColor': u'C28709',
  u'dominantColorBackground': u'Brown',
  u'dominantColorForeground': u'Brown',
  u'dominantColors': [u'Brown', u'Yellow'],
  u'isBWImg': False},
 u'description': {u'captions': [{u'confidence': 0.4430294665741739,
    u'text': u'a chair sitting in front of a beach'}],
  u'tags': [u'outdoor',
   u'water',
   u'beach',
   u'sunset',
   u'ocean',
   u'sitting',
   u'chair',
   u'sun',
   u'sand',
   u'front',
   u'set',
   u'board',
   u'man',
   u'umbrella',
   u'surfing',
   u'standing']},
 u'faces': [],
 u'metadata': {u'format': u'Jpeg', u'height': 1080, u'width': 1920},
 u'requestId': u'eb75b3c1-c4c1-42d6-885a-def7812bba3d'}

In [104]:
res['description']['captions'][0]['text']

u'a chair sitting in front of a beach'

In [106]:
res['categories'][0]['name']

u'outdoor_oceanbeach'

In [112]:
type(res['description']['tags'])

list

In [114]:
res['description']['tags'][:5]

[u'outdoor', u'water', u'beach', u'sunset', u'ocean']